In [1]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
%matplotlib inline  
%load_ext Cython

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
import optuna

C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
# data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
data_train_path="data_train.csv"
data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [3]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
user_map = {i : user for i, user in enumerate(data_target["user_id"])}
item_map_inv = {item : i for i, item in item_map.items()}
user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

<13024x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [4]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2617 (20.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2585 (19.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2538 (19.5%) Users that have less than 1 test interactions


In [7]:
def train_evaluate(optuna_trial):
    topk = optuna_trial.suggest_int("topK", 1, 100)
    alpha = optuna_trial.suggest_float("alpha", 0.0, 0.8)
    beta = optuna_trial.suggest_float("beta", 0.0, 0.3)

    mAP = 0.0
    for i in range(0, test_folds):
        RP3beta = RP3betaRecommender(URM_trains[i], verbose=False)
        RP3beta.fit(topK=topk, alpha=alpha, beta=beta)
        result_df, _ = evaluator_tests[i].evaluateRecommender(RP3beta)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [9]:
study = optuna.create_study(direction="maximize")
study.optimize(train_evaluate, n_trials=400)

[I 2023-12-11 15:01:24,517] A new study created in memory with name: no-name-255d64c0-f7f2-4635-9b58-a93091fa9647


EvaluatorHoldout: Processed 10407 (100.0%) in 8.16 sec. Users per second: 1276
EvaluatorHoldout: Processed 10439 (100.0%) in 7.78 sec. Users per second: 1342
EvaluatorHoldout: Processed 10486 (100.0%) in 7.39 sec. Users per second: 1420


[I 2023-12-11 15:02:14,871] Trial 0 finished with value: 0.047285406574189344 and parameters: {'topK': 76, 'alpha': 0.5698291254483303, 'beta': 0.05326546889291669}. Best is trial 0 with value: 0.047285406574189344.


EvaluatorHoldout: Processed 10407 (100.0%) in 7.29 sec. Users per second: 1428
EvaluatorHoldout: Processed 10439 (100.0%) in 7.58 sec. Users per second: 1378
EvaluatorHoldout: Processed 10486 (100.0%) in 7.43 sec. Users per second: 1412


[I 2023-12-11 15:03:00,240] Trial 1 finished with value: 0.04646899851030483 and parameters: {'topK': 52, 'alpha': 0.004648691446542408, 'beta': 0.2856771215760075}. Best is trial 0 with value: 0.047285406574189344.


...

[I 2023-12-11 19:35:10,256] Trial 397 finished with value: 0.048755040263111504 and parameters: {'topK': 41, 'alpha': 0.2798797258349148, 'beta': 0.16401640857871858}. Best is trial 195 with value: 0.049227550567598775.


EvaluatorHoldout: Processed 10407 (100.0%) in 6.43 sec. Users per second: 1619
EvaluatorHoldout: Processed 10439 (100.0%) in 6.36 sec. Users per second: 1642
EvaluatorHoldout: Processed 10486 (100.0%) in 6.42 sec. Users per second: 1632


[I 2023-12-11 19:35:46,428] Trial 398 finished with value: 0.04897952603024783 and parameters: {'topK': 21, 'alpha': 0.22660644399997015, 'beta': 0.1788036402983109}. Best is trial 195 with value: 0.049227550567598775.


EvaluatorHoldout: Processed 10407 (100.0%) in 6.45 sec. Users per second: 1615
EvaluatorHoldout: Processed 10439 (100.0%) in 6.45 sec. Users per second: 1618
EvaluatorHoldout: Processed 10486 (100.0%) in 6.46 sec. Users per second: 1623


[I 2023-12-11 19:36:24,261] Trial 399 finished with value: 0.04899765634332511 and parameters: {'topK': 32, 'alpha': 0.4762532763302827, 'beta': 0.18579796518431282}. Best is trial 195 with value: 0.049227550567598775.


In [13]:
# [I 2023-12-11 12:18:43,638] Trial 62 finished with value: 0.0489882573806513 and parameters: {'topK': 32, 'alpha': 0.2313857839079508, 'beta': 0.20420155472511597}. Best is trial 62 with value: 0.0489882573806513.
study.best_trial

FrozenTrial(number=195, state=1, values=[0.049227550567598775], datetime_start=datetime.datetime(2023, 12, 11, 17, 17, 11, 833446), datetime_complete=datetime.datetime(2023, 12, 11, 17, 17, 53, 423595), params={'topK': 32, 'alpha': 0.22028996694465927, 'beta': 0.17339063177346345}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'topK': IntDistribution(high=100, log=False, low=1, step=1), 'alpha': FloatDistribution(high=0.8, log=False, low=0.0, step=None), 'beta': FloatDistribution(high=0.3, log=False, low=0.0, step=None)}, trial_id=195, value=None)

In [ ]:
# params={'topK': 32, 'alpha': 0.22028996694465927, 'beta': 0.17339063177346345} values=[0.049227550567598775]